In [76]:
import os
import pandas as pd

datapath = "../data/raw"

In [77]:
df1 = pd.read_csv(os.path.join(datapath, "chembl_uniprot_mapping.txt"), sep="\t")
df2 = pd.read_csv(os.path.join(datapath, "mtuberculosis_ChEMBL_data.csv"))

In [78]:
df2.columns

Index(['activity_id', 'assay_id', 'assay_chembl_id', 'assay_type',
       'assay_confidence_score', 'assay_organism', 'doc_chembl_id', 'tid',
       'target_type', 'target_organism', 'target_chembl_id', 'target_tax_id',
       'compound_chembl_id', 'canonical_smiles', 'MW', 'pchembl',
       'activity_comment', 'standard_text', 'value', 'unit', 'activity_type',
       'relation', 'pchembl_calculated'],
      dtype='object')

In [79]:
# Is binding for proteins and functional for organism?
print("Functional",  df2[df2["assay_type"]=="F"]["target_type"].value_counts())
print("Binding", df2[df2["assay_type"]=="B"]["target_type"].value_counts())

Functional target_type
SINGLE PROTEIN     217253
ORGANISM           211230
UNCHECKED          200094
PROTEIN COMPLEX     69290
NON-MOLECULAR         273
SUBCELLULAR             8
Name: count, dtype: int64
Binding target_type
SINGLE PROTEIN     8932
UNCHECKED          5999
PROTEIN COMPLEX     589
ORGANISM            328
SUBCELLULAR          30
PROTEIN FAMILY        1
Name: count, dtype: int64


In [81]:
df_ = df2[df2["target_type"]=="PROTEIN COMPLEX"]
df_["target_chembl_id"].nunique()

3

In [29]:
# Unchecked counts
df_ = df2[df2["target_type"]=="UNCHECKED"]
df_ = df_["assay_chembl_id"].value_counts()
df_ = df_.reset_index()
df_.columns = ["assay_chembl_id", "counts"]
df_.to_csv("../data/processed/chembl/chembl_unchecked.csv", index=False)

In [91]:
#manual curation 
# DO NOT DELETE
df = pd.read_csv("../data/processed/chembl/chembl_unchecked_manual.csv")
df = df[~df["target_manual"].isna()]
df = df[["assay_chembl_id", "target_chembl_id"]].drop_duplicates(keep="first")

In [92]:
#mapping from df to the mtuberculosis_ChEMBL_data
df2 = pd.read_csv(os.path.join(datapath, "mtuberculosis_ChEMBL_data.csv"))
lookup = df.set_index("assay_chembl_id")["target_chembl_id"]
df2["target_chembl_id"] = df2["assay_chembl_id"].map(lookup)

In [93]:
df2.to_csv("../data/processed/chembl/mtuberculosis_chembl_edited.csv", index=False)

# Generate merging file target_chembl_id to uniprot_ac

In [94]:
df1 = pd.read_csv(os.path.join(datapath, "chembl_uniprot_mapping.txt"), sep="\t")
df2 = pd.read_csv("../data/raw/mtuberculosis_ChEMBL_data.csv")
targets = list(set(df2["target_chembl_id"].tolist()))
len(targets)

146

In [95]:
df_ = (
    df1[df1["chembl_id"].isin(targets)]
      .dropna(subset=["uniprot_ac"])                   
      .rename(columns={"chembl_id": "target_chembl_id"})
)
df_["target_chembl_id"].value_counts()

target_chembl_id
CHEMBL2364166    8
CHEMBL4662931    2
CHEMBL3430898    2
CHEMBL3883301    2
CHEMBL2361       1
                ..
CHEMBL6086       1
CHEMBL6070       1
CHEMBL6069       1
CHEMBL6067       1
CHEMBL3988585    1
Name: count, Length: 136, dtype: int64

In [96]:
#add manual targets
df = pd.read_csv("../data/processed/chembl/chembl_unchecked_manual.csv")
df = df[~df["uniprot_ac"].isna()]
df = df[df["uniprot_ac"]!="UNKNOWN"]
df=df[["target_chembl_id","uniprot_ac"]]

In [97]:
df_uniprots = list(set(df["uniprot_ac"].tolist()))
df_uniprots2 = list(set(df_["uniprot_ac"].tolist()))

In [98]:
print(len(df_uniprots), len(df_uniprots2), len(set(df_uniprots+df_uniprots2)))

32 144 154


In [99]:
uniprot_ac_in_chembl = pd.DataFrame({"uniprot_ac":list(set(df_uniprots+df_uniprots2))})
uniprot_ac_in_chembl.to_csv(("../data/processed/chembl/uniprot_ac_in_chembl.csv"), index=False)